In [ ]:
import pyesapi
import atexit
import time
import os
app = pyesapi.CustomScriptExecutable.CreateApplication('python_demo')
atexit.register(app.Dispose);

In [ ]:
import pandas as pd


def d_at_v(plan,structure,volume):
    _dose=plan.GetDoseAtVolume(structure,volume,
                               pyesapi.VolumePresentation.Relative,
                               pyesapi.DoseValuePresentation.Absolute)
    return _dose.Dose

columns = (
    'PatientId',
    'PlanId',
    'StructureId',
    'DicomType',
    'Volume(cc)',
    'IsHighRes',
    'D95%(Gy)',
    'D25%(Gy)',
    'D50%(Gy)'
)

def get_structure_and_dose_info(plan, get_volumes=True, get_dose=True):
    returning_list = []
    for s in plan.StructureSet.Structures:
        structure_list = [
        plan.StructureSet.Patient.Id,
        plan.Id,
        s.Id,
        s.DicomType,
        ]
        if get_volumes:
            structure_list += [
                s.Volume,
                s.IsHighResolution
            ]
        else:
            structure_list += [
                'NA',
                'NA'
            ]
        if get_dose:
            structure_list += [
                d_at_v(plan,s,95),
                d_at_v(plan,s,25),
                d_at_v(plan,s,50)]
        else:
            structure_list += [
                'NA', 'NA', 'NA'
            ]
        returning_list.append(tuple(structure_list))
    return returning_list

In [ ]:
fid = open(os.path.join('.', 'PatientID'))
line = fid.readline()
fid.close()
MRN = line.strip('\n')
app.ClosePatient()
patient = app.OpenPatientById(MRN)
plan = patient.CoursesLot(0).PlanSetupsLot(0)

In [ ]:
plans = [i for i in patient.CoursesLot(0).PlanSetups]

In [ ]:
[i.StructureSet for i in plans]

In [ ]:
def returndataframe_list():
    dataframe_list = []
    patients_checked = []
    get_volumes = False
    global_get_dose = False
    i = 0
    for pat_sum in app.PatientSummaries:
        time.sleep(0.0)
        i += 1
        if i > 100:
            break
        MRN = pat_sum.Id
        print('MRN: {}'.format(MRN))
        if MRN in patients_checked:
            continue
        patients_checked.append(MRN)
        app.ClosePatient()
        patient = app.OpenPatientById(MRN)
        for course in patient.Courses:
            time.sleep(0.0)
            print('Course: {}'.format(course.Id))
            if course.Id.find('QA') != -1 or course.Id.find('use') != -1:
                continue
            print(plans)
            for plan in course.PlanSetups:
                time.sleep(0.0)
                print(i)
                print('Plan: {}'.format(plan.Id))
                if plan.StructureSet is None: # No structures present, move on
                    continue
                if global_get_dose and plan.Dose is not None:
                    get_dose = True
                else:
                    get_dose = False
                dataframe_list.append(pd.DataFrame(get_structure_and_dose_info(plan, get_volumes=get_volumes, get_dose=get_dose),
                                                   columns=columns))
    return dataframe_list
dataframe_list = returndataframe_list()

In [ ]:
structure_dataframe = pd.concat(dataframe_list,ignore_index=True)
structure_dataframe

In [ ]:
structure_dataframe.to_excel(os.path.join('.', 'StructureData.xlsx'), index=False)

In [ ]:
# if you insist
with open('./StructureData.csv','w') as f:
    f.write(structure_dataframe.to_csv(sep=','))

In [ ]:
i = 0
for pat_sum in app.PatientSummaries:
    print(i)
    i += 1
    if i > 200:
        break
    MRN = pat_sum.Id
    if MRN in patients_checked:
        continue
    patients_checked.append(MRN)
    app.ClosePatient()
    patient = app.OpenPatientById(MRN)
    courses = patient.Courses
    for course in courses:
        if course.Id.find('QA') == -1:
            plans = [i for i in course.PlanSetups]
            for plan in plans:
                if plan.StructureSet is None: # No structures present, move on
                    continue
                for structure in plan.StructureSet.Structures:
                    structure_name = structure.Id
                    if structure_name not in structures:
                        structures[structure_name] = {'PatientID': [], 'Course': [], 'Plan': []}
                    structures[structure_name]['PatientID'].append(MRN)
                    structures[structure_name]['Course'].append(course.Id)
                    structures[structure_name]['Plan'].append(plan.Id)

In [ ]:
structures['Lung RT']

In [ ]:
list(structures.keys())

In [ ]:
structures.keys()

In [ ]:
patient.CoursesLot('C1').PlanSetupsLot(0)